# Hydrafloods

## Ejemplo Huallaga

### Importando librerias e información

In [1]:
# import the hydrafloods and ee package
import hydrafloods as hf
import ee
ee.Initialize()

In [2]:
import os
directory = os.path.abspath('.') 
print(directory) 

d:\Hydraflood\Huallaga


#### Eventos historicos
Se identificaron por periodos, para analizar la variabildad espacio-temporal.

In [3]:
# Inundaciones en Tingo María
# https://elcomercio.pe/peru/huanuco/lluvias-tingo-maria-dejan-cien-familias-damnificadas-295832-noticia/ no se encontraron imagenes para la fecha de 2014

# Sentinel 1 tiene información desde 03 de abril de 2014
# specify start and end time as well as geographic region to process
#Evento 1
#start_time = "2015-04-10"
#end_time =  "2015-10-01"
#Evento 2
#start_time = "2016-01-01"
#end_time =  "2016-03-31"
#Evento 3
#start_time = "2016-04-01"
#end_time =  "2016-10-31"
#Evento 4
# https://elpopular.pe/actualidad-y-policiales/259568-huanuco-rio-desborda-intensas-lluvias-desaparece-joven-tingo-maria-policia-video
start_time = "2021-02-10"
end_time =  "2021-02-28"

# region = ee.Geometry.Rectangle([-76.024891,  -9.343222, -75.983438,  -9.275245 ])
region = ee.Geometry.Rectangle([-76.030416,  -9.343747, -75.975255,  -9.271063 ])

In [4]:
# get the Sentinel-1 collection
# the hf.dataset classes performs the spatial-temporal filtering for you
s1 = hf.datasets.Sentinel1(region, start_time, end_time)
type(s1)

hydrafloods.datasets.Sentinel1

#### "Edge Otsu" algorithm

La aplicación HYDRAFloods incluye dentro de su modulo de umbrales 3 algoritmos genericos de mapeo de agua superficial:
* Edge Otsu: hydrafloods.edge_otsu ([Donchyts et al., 2016](https://doi.org/10.3390/rs8050386); [Markert et al., 2020](https://doi.org/10.3390/rs12152469))
* Bmax Otsu: hydrafloods.bmax_otsu ([Cao et al., 2019](https://doi.org/10.3390/w11040786); [Markert et al., 2020](https://doi.org/10.3390/rs12152469))
* Extensión de KMeans: hydrafloods.kmeans_extent([Chang et al., 2020](https://doi.org/10.1016/j.rse.2020.111732))

In [5]:
# apply a water mapping function to the S1 dataset
# this applies the "Edge Otsu" algorithm from https://doi.org/10.3390/rs12152469
water_imgs = s1.apply_func(
    hf.thresholding.edge_otsu,
    initial_threshold=-14,
    thresh_no_data=-20,
    edge_buffer=300
)
type(water_imgs)

hydrafloods.datasets.Sentinel1

In [6]:
# take the mode from multiple images
# since this is just imagery from one day, it will simply mosaic the images
water_map = ee.Image(water_imgs.collection.mode())
water_map

##### Mapa EE

In [7]:
#!pip install geemap
import geemap
#water_map

In [8]:
# Create a default map
Map = geemap.Map()

#Load an image.
image = water_map #ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
#Create an NDWI image, define visualization parameters and display.
#ndwi = image.normalizedDifference(['B3', 'B5'])
ndwiViz = {'min': 0,
           'max': 1,
           'palette': ['F2F2F2','fae7b5','318ce7','E6E600','E9BD3A','ECB176','EFC2B3','F2F2F2','318ce7'],
           }
Map.centerObject(region,12)  #setCenter(-122.1899, 37.5010, 10) # San Francisco Bay
Map.addLayer(image,
             ndwiViz,
            'HYDRAFloods',
            shown=True,
            opacity=0.7
             )#, False)

# Display the map
Map

Map(center=[-9.307404800192744, -76.00283550000388], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
#Load an image.
image = water_map #ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# https://github.com/giswqs/earthengine-py-notebooks/blob/master/Image/image_visualization.ipynb

# Request a default thumbnail of the DEM with defined linear stretch.
# Set masked pixels (ocean) to 1000 so they map as gray.
thumbnail1 = image.getThumbURL({
  'min': 0,
  'max': 1,
  'dimensions': 500,
  'crs': 'EPSG:32718',
  'palette': ['F2F2F2','fae7b5','318ce7','E6E600','E9BD3A','ECB176','EFC2B3','F2F2F2','318ce7'],
  'region': region,
  #'region': ee.Geometry.Rectangle([103, 11.4, 107, 12.6 ])
})
print('Default extent and size:', thumbnail1)

Default extent and size: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/f3d745af540ab153e287b96f43b0ac1b-574014da61aba22fb81807512eb1643f:getPixels


##### Exportando a Asset de GEE

In [11]:
# export the water map
hf.geeutils.export_image(
    water_map,
    region,
    #"users/<YOUR_USERNAME>/water_map_example",
    asset_id="users/20200782/hydrafloods_output/water_map_huallaga2016", #El username es el nombre de usuario de GEE
    export_type='toAsset',
    scale=30,
)
# Se demora en verlo reflejado en los Assets de GEE https://code.earthengine.google.com/
#Hora de inicio 9:02

##### Exportando a Drive

In [12]:
# export the water map
hf.geeutils.export_image(
    water_map,
    region,
    #"users/<YOUR_USERNAME>/water_map_example",
    description="water_map_huallaga2016", #El username es el nombre de usuario de GEE
    export_type='toDrive',
    scale=30,
    folder="GEE_SENAMHI"
)
# Se demora en verlo reflejado en los Assets de GEE https://code.earthengine.google.com/
#Hora de inicio 3:18

### "Fuzzy Otsu" Algorithm Model

In [10]:
# apply a water mapping function to the S1 dataset
# this applies the "Edge Otsu" algorithm from https://doi.org/10.3390/rs12152469
water_imgs_fuzzy_outsu = s1.apply_func(
    hf.thresholding.fuzzy_otsu,
    initial_threshold=-15.5,
)
water_imgs_fuzzy_outsu

HYDRAFloods Dataset:
{'asset_id': 'COPERNICUS/S1_GRD',
 'end_time': '2021-02-28',
 'name': 'Sentinel1',
 'region': [[[...], [...], [...], [...], [...]]],
 'start_time': '2021-02-10'}

In [11]:
water_map_fuzzy_outsu = ee.Image(water_imgs_fuzzy_outsu.collection.mode())
water_map_fuzzy_outsu

In [15]:
#Load an image.
image = water_map_fuzzy_outsu #ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# https://github.com/giswqs/earthengine-py-notebooks/blob/master/Image/image_visualization.ipynb

# Request a default thumbnail of the DEM with defined linear stretch.
# Set masked pixels (ocean) to 1000 so they map as gray.
thumbnail1 = image.getThumbURL({
  'min': 0,
  'max': 1,
  'dimensions': 500,
  'crs': 'EPSG:32718',
  'palette': ['fae7b5','318ce7','E6E600','E9BD3A','ECB176','EFC2B3','F2F2F2'],
  'region': region #ee.Geometry.Rectangle([-84.6, -55.9, -32.9, 15.7]),
  #'region': ee.Geometry.Rectangle([103, 11.4, 107, 12.6 ])
})
print('Default extent and size:', thumbnail1)

Default extent and size: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/40080c55e84d7f43e7143a66372753df-61a052fbb028a6303d5b28959836e69b:getPixels


In [12]:
# Create a default map
Map = geemap.Map()

#Load an image.
#image = water_map_fuzzy_outsu #ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
#Create an NDWI image, define visualization parameters and display.
#ndwi = image.normalizedDifference(['B3', 'B5'])
ndwiViz = {'min': 0,
           'max': 1,
           'palette': ['F2F2F2','fae7b5','318ce7','E6E600','E9BD3A','ECB176','EFC2B3','F2F2F2','318ce7'],
           }
Map.centerObject(region,12)  #setCenter(-122.1899, 37.5010, 10) # San Francisco Bay
Map.addLayer(water_map_fuzzy_outsu,
             ndwiViz,
            'HYDRAFloods',
            shown=True,
            opacity=0.7
             )#, False)

# Display the map
Map

Map(center=[-9.307404800192744, -76.00283550000388], controls=(WidgetControl(options=['position', 'transparent…